In [ ]:
!gdown --id 18w7LxJf1ueHAc28srzwhXeyLf6bHCU4B

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=18w7LxJf1ueHAc28srzwhXeyLf6bHCU4B
From (redirected): https://drive.google.com/uc?id=18w7LxJf1ueHAc28srzwhXeyLf6bHCU4B&confirm=t&uuid=98f1ab4a-f7cc-48dc-8c16-59e9619b63c8
To: /content/DataSets.zip
100% 118M/118M [00:01<00:00, 59.5MB/s]


In [ ]:
!unzip /content/DataSets.zip

Archive:  /content/DataSets.zip
   creating: DataSets/Clean/
  inflating: DataSets/Clean/1.jpg    
  inflating: DataSets/Clean/10.jpeg  
  inflating: DataSets/Clean/100.jpg  
  inflating: DataSets/Clean/101.jpg  
  inflating: DataSets/Clean/102.jpg  
  inflating: DataSets/Clean/103.jpg  
  inflating: DataSets/Clean/104.jpg  
  inflating: DataSets/Clean/105.jpg  
  inflating: DataSets/Clean/106.jpg  
  inflating: DataSets/Clean/107.jpg  
  inflating: DataSets/Clean/108.jpg  
  inflating: DataSets/Clean/109.jpg  
  inflating: DataSets/Clean/11.jpeg  
  inflating: DataSets/Clean/110.jpg  
  inflating: DataSets/Clean/111.jpg  
  inflating: DataSets/Clean/112.jpg  
  inflating: DataSets/Clean/113.jpg  
  inflating: DataSets/Clean/114.jpg  
  inflating: DataSets/Clean/115.jpg  
  inflating: DataSets/Clean/116.jpg  
  inflating: DataSets/Clean/117.jpg  
  inflating: DataSets/Clean/118.jpg  
  inflating: DataSets/Clean/119.jpg  
  inflating: DataSets/Clean/12.jpeg  
  inflating: DataSets/Clean

In [ ]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
!pip install pyheif
import pyheif
!pip install imagehash
import imagehash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 6.5 MB/s eta 0:00:00


In [ ]:



def delete_similar_images(image_directory, similarity_threshold=5):
    """
    Deletes similar images in a given directory based on perceptual hashing.

    :param image_directory: The directory containing the images to process.
    :param similarity_threshold: The maximum difference in hashes to consider images as similar.
                                  Lower values mean stricter similarity detection.
    """
    # Dictionary to store image hashes
    hash_dict = {}

    # Loop through all images in the directory
    for image_filename in os.listdir(image_directory):
        if image_filename.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp')):
            image_path = os.path.join(image_directory, image_filename)
            try:
                # Open the image and compute its hash
                with Image.open(image_path) as img:
                    img_hash = imagehash.average_hash(img)

                # Check if the hash is in the dictionary (i.e., if a similar image was already processed)
                similar_images = [key for key, value in hash_dict.items() if img_hash - value <= similarity_threshold]
                if similar_images:
                    # If a similar image is found, delete the current image
                    os.remove(image_path)
                    print(f"Deleted {image_path} as it is similar to {similar_images[0]}")
                else:
                    # Otherwise, add the image hash to the dictionary
                    hash_dict[image_filename] = img_hash
            except Exception as e:
                print(f"Error processing {image_filename}: {e}")

    # Print completion message
    print("Finished processing images.")

# Example usage:
# delete_similar_images('/path/to/image/dataset', similarity_threshold=5)

In [ ]:


def process_and_augment_images(input_dir, output_dir, size=(512, 512), augmentation_factor=5):
    """
    Process images by resizing and then apply augmentations. Original images are also saved.
    Augmented images will be saved in color.
    """
    datagen = ImageDataGenerator(
        rescale=1./255,  # Rescale the image values to [0, 1]
        rotation_range=30,  # Degree range for random rotations
        width_shift_range=0.2,  # Range for random horizontal shifts
        height_shift_range=0.2,  # Range for random vertical shifts
        shear_range=0.15,  # Shear intensity (Shear angle in counter-clockwise direction)
        zoom_range=0.2,  # Range for random zoom
        horizontal_flip=True,  # Randomly flip inputs horizontally
        fill_mode='nearest'  # Points outside the boundaries are filled according to the nearest mode
    )

    for class_name in os.listdir(input_dir):
        class_dir = os.path.join(input_dir, class_name)
        output_class_dir = os.path.join(output_dir, class_name)

        if not os.path.exists(output_class_dir):
            os.makedirs(output_class_dir)

        for file_name in os.listdir(class_dir):
            if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.heic')):
                file_path = os.path.join(class_dir, file_name)

                if file_name.lower().endswith('.heic'):
                    image = pyheif.read(file_path)
                    img = Image.frombytes(image.mode, image.size, image.data)
                else:
                    img = Image.open(file_path).convert('RGB')

                img_resized = img.resize(size)
                img_resized.save(os.path.join(output_class_dir, file_name))  # Save the resized original image

                img_array = np.array(img_resized).reshape(-1, size[0], size[1], 3)  # Ensure the image is in color

                i = 0
                for batch in datagen.flow(img_array, batch_size=1, save_to_dir=output_class_dir, save_prefix='aug_', save_format='jpeg'):
                    i += 1
                    if i >= augmentation_factor:
                        break


In [ ]:


def rename_images_in_directory(directory_path):
    """
    Renames image files in the specified directory to a sequential numbering while retaining their original extension.

    Args:
    - directory_path: The path to the directory containing the image files to be renamed.

    Returns:
    None
    """
    # Get a list of all files in the directory
    files = os.listdir(directory_path)

    # Iterate through the files and rename them
    i = 1
    for img in files:
        # Check if the file is an image and extract the extension
        if img.lower().endswith(('.png', '.jpg', '.jpeg', '.heic')):
            extension = os.path.splitext(img)[1]  # Extract the file extension

            # Create a new filename with the number i and original extension
            new_filename = f"{i}{extension}"

            # Construct the full path for the old and new files
            old_path = os.path.join(directory_path, img)
            new_path = os.path.join(directory_path, new_filename)

            # Rename the file
            os.rename(old_path, new_path)

            # Increment the counter
            i += 1

    # Print a message indicating that the files have been renamed
    print("Files renamed successfully.")

# Example usage
clean = '/content/DataSets/Clean'
leaf = '/content/DataSets/leaves'
rename_images_in_directory(clean)
rename_images_in_directory(leaf)



Files renamed successfully.
Files renamed successfully.


In [ ]:
delete_similar_images(clean, similarity_threshold=5)
delete_similar_images(leaf, similarity_threshold=5)

Finished processing images.
Finished processing images.


In [ ]:
def split_dataset(data_dir, test_size=0.2, val_size=0.2):
    """
    Split the dataset into training, validation, and test sets.
    """
    file_paths = []
    labels = []

    for label in os.listdir(data_dir):
        label_dir = os.path.join(data_dir, label)
        if os.path.isdir(label_dir):
            for file_name in os.listdir(label_dir):
                file_paths.append(os.path.join(label_dir, file_name))
                labels.append(label)

    train_paths, temp_paths, train_labels, temp_labels = train_test_split(file_paths, labels, test_size=test_size, random_state=42)

    val_size_adjusted = val_size / (1 - test_size)
    val_paths, test_paths, val_labels, test_labels = train_test_split(temp_paths, temp_labels, test_size=val_size_adjusted, random_state=42)

    return train_paths, val_paths, test_paths, train_labels, val_labels, test_labels



In [ ]:
raw_images_dir = '/content/DataSets'
augmented_images_dir = '/content/Preprocesed_image'





In [ ]:
# Process and augment images
process_and_augment_images(raw_images_dir, augmented_images_dir)

In [ ]:
# Split the dataset
train_paths, val_paths, test_paths, train_labels, val_labels, test_labels = split_dataset(augmented_images_dir)

In [ ]:


# Output the results
print(f"Training set size: {len(train_paths)}")
print(f"Validation set size: {len(val_paths)}")
print(f"Test set size: {len(test_paths)}")

Training set size: 822
Validation set size: 154
Test set size: 52


In [ ]:
print(train_paths)

['/content/Preprocesed_image/Clean/34.jpg', '/content/Preprocesed_image/Clean/aug__0_5189.jpeg', '/content/Preprocesed_image/Clean/aug__0_2853.jpeg', '/content/Preprocesed_image/Clean/aug__0_6217.jpeg', '/content/Preprocesed_image/Clean/aug__0_7118.jpeg', '/content/Preprocesed_image/leaves/aug__0_8251.jpeg', '/content/Preprocesed_image/leaves/aug__0_8678.jpeg', '/content/Preprocesed_image/Clean/aug__0_6389.jpeg', '/content/Preprocesed_image/Clean/189.jpeg', '/content/Preprocesed_image/leaves/58.jpg', '/content/Preprocesed_image/leaves/aug__0_5558.jpeg', '/content/Preprocesed_image/Clean/aug__0_7261.jpeg', '/content/Preprocesed_image/leaves/aug__0_2459.jpeg', '/content/Preprocesed_image/leaves/aug__0_2799.jpeg', '/content/Preprocesed_image/Clean/aug__0_6655.jpeg', '/content/Preprocesed_image/Clean/aug__0_367.jpeg', '/content/Preprocesed_image/leaves/aug__0_4961.jpeg', '/content/Preprocesed_image/Clean/aug__0_2155.jpeg', '/content/Preprocesed_image/Clean/aug__0_6393.jpeg', '/content/Prep

In [ ]:
!zip -r /content/Preprocesed_image.zip /content/Preprocesed_image

  adding: content/Preprocesed_image/ (stored 0%)
  adding: content/Preprocesed_image/Clean/ (stored 0%)
  adding: content/Preprocesed_image/Clean/aug__0_2580.jpeg (deflated 0%)
  adding: content/Preprocesed_image/Clean/aug__0_2169.jpeg (deflated 3%)
  adding: content/Preprocesed_image/Clean/aug__0_7420.jpeg (deflated 0%)
  adding: content/Preprocesed_image/Clean/aug__0_681.jpeg (deflated 0%)
  adding: content/Preprocesed_image/Clean/aug__0_6788.jpeg (deflated 0%)
  adding: content/Preprocesed_image/Clean/aug__0_9333.jpeg (deflated 0%)
  adding: content/Preprocesed_image/Clean/aug__0_3810.jpeg (deflated 0%)
  adding: content/Preprocesed_image/Clean/173.jpg (deflated 1%)
  adding: content/Preprocesed_image/Clean/aug__0_9743.jpeg (deflated 8%)
  adding: content/Preprocesed_image/Clean/aug__0_7253.jpeg (deflated 0%)
  adding: content/Preprocesed_image/Clean/81.jpg (deflated 0%)
  adding: content/Preprocesed_image/Clean/12.jpeg (deflated 0%)
  adding: content/Preprocesed_image/Clean/aug__0_